In [1]:
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
from ipywidgets import widgets
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd

In [2]:
init_notebook_mode(connected=True)

In [3]:
visualizations_data=pd.read_csv("./data/"+"all_predicted.csv")
visualizations_data.head()

,Unnamed: 0,height_in,weight_lbs,Age,season_num,G,MP,PER,WS,WS/48,...,VORP,Pos_C,Pos_PF,Pos_PG,Pos_SF,Pos_SG,season_categorical,salary,season_start,player_name
0,0,78.000000,200.000000,23.000000,1.000000,68.000000,15.500000,10.100000,2.100000,0.096000,...,-0.100000,0.000000,-2.775558e-17,0.000000,0.000000,1.00000,32.000000,5.994764e+06,2016,Álex Abrines
1,1,78.000000,200.000000,24.000000,2.000000,75.000000,15.100000,9.000000,2.200000,0.094000,...,-0.100000,0.000000,-2.775558e-17,0.000000,0.000000,1.00000,33.000000,5.725000e+06,2017,Álex Abrines
2,2,78.000000,200.000000,25.000000,3.000000,31.000000,19.000000,6.300000,0.600000,0.053000,...,-0.200000,0.000000,-2.775558e-17,0.000000,0.000000,1.00000,34.000000,5.455236e+06,2018,Álex Abrines
3,3,78.116072,200.476044,26.046765,4.297592,49.894572,17.528521,9.068572,1.097364,0.059656,...,0.012040,0.015134,4.637657e-04,0.048036,0.240926,0.68970,36.048778,4.360653e+06,2019,Álex Abrines
4,4,78.454765,197.012272,26.104838,5.111217,51.797230,17.928437,9.078428,0.892712,0.051887,...,-0.034448,0.018093,3.667988e-02,0.076530,0.337691,0.53417,39.442324,4.823157e+06,2020,Álex Abrines


In [4]:
visualizations_data=visualizations_data.drop(["Unnamed: 0"],axis=1)

In [5]:
# Dropdown Widget Creation for Player Selection

player_names = widgets.Dropdown(
    options=list(np.unique(visualizations_data['player_name'])),
    value=visualizations_data.player_name.values[0],
    description='Player :',
)


In [6]:
def validate():
    if player_names.value in np.unique(visualizations_data['player_name']):
        return True
    else:
        return False

In [7]:
## Create Baseline Dashboard for display of Player Salary Predictions
# Salary Bar Line Results
trace1 = go.Bar(x=visualizations_data.loc[visualizations_data["player_name"]==visualizations_data.player_name.values[0],'season_start'], 
                y=visualizations_data.loc[visualizations_data["player_name"]==visualizations_data.player_name.values[0],'salary'], 
                name='Salary per year')

# Player Attributes for display
selected_cols=["player_name","height_in","weight_lbs","Age"]
trace2 = go.Table(
        header=dict(
            values=["Player<br>Name", "height<br>(inches)", "weight<br>(lbs)","Age"],
            font=dict(size=10),
            align=['left'] * 5
        ),
    columnwidth=[0.5,0.3,0.3,0.2,0.2],
        cells=dict(
            values=[visualizations_data.loc[ visualizations_data["player_name"]==visualizations_data.player_name.values[0],col][2:3].tolist() for col in selected_cols],
            align = ['left'] * 5,
            height = 20
        
        )

    )

fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.5, 0.5],
    horizontal_spacing=.1,
    specs=[[{"type": "table"},{"type": "bar" }]]
)


# Salary Dotted Line Trace
trace1_line=go.Scatter(x=visualizations_data.loc[visualizations_data["player_name"]==visualizations_data.player_name.values[0],'season_start'], 
                y=visualizations_data.loc[visualizations_data["player_name"]==visualizations_data.player_name.values[0],'salary'], 
                         name='Dotted Line',
            line = dict(color='firebrick', width=4, dash='dot'),
            marker = dict(size=7 )
                      )

trace1_divider = go.Scatter(
    x=[.5 + visualizations_data.loc[visualizations_data["player_name"]==visualizations_data.player_name.values[0], "season_start"].values[2], .5 + visualizations_data.loc[visualizations_data["player_name"]==visualizations_data.player_name.values[0], "season_start"].values[2]] , 
    y=[-0.05*visualizations_data.loc[visualizations_data["player_name"]==visualizations_data.player_name.values[0],'salary'].max() ,  visualizations_data.loc[visualizations_data["player_name"]==visualizations_data.player_name.values[0],'salary'].max()  ],
    line_color='White',
    mode= "lines", 
    name='Actual and Predicted Salaries Dividing line',
    showlegend=False
)

fig.add_trace(trace1, row=1, col=2)
fig.add_trace(trace1_line, row=1, col=2)
fig.add_trace(trace1_divider, row=1, col=2)


fig.add_trace(trace2,  row=1, col=1)
fig.layout.legend =dict(x=.5, y=1.1)

g = go.FigureWidget(data=fig,
                    layout=go.Layout(
                        title=dict(
                            text='BASKETBALL PLAYER SALARIES'
                        )                
                    )          
                   )

g.update_layout(
    yaxis=dict(title="Salary"),
    xaxis=dict(title="Years"),
    template="plotly_dark",
    margin=dict(r=30, t=45, b=40, l=30),
    annotations=[
        go.layout.Annotation(
            text="BASKETBALL PLAYER'S DATA",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0,
            y=0),
        
        
        go.layout.Annotation(
            text="5 year Market Value :  $ "+str(round(visualizations_data.loc[visualizations_data["player_name"]==visualizations_data.player_name.values[0], "salary"].values[-1-6:-1-1].sum()/ 1000000,2))+" M",
            showarrow=False,
            xref="paper",
            yref="paper",
            font=dict(family='Arial',size=16),
            x=0,
            y=.7)
    ]
)


g.show()

In [8]:
# Add dynamic response layer to update figure upon player selection
def response(change):
    if validate():
        #print ("\n\nplayer_names.value: ",player_names.value,"\n\n")
        x1 = visualizations_data.loc[visualizations_data["player_name"]==player_names.value,'season_start']
        y1 = visualizations_data.loc[visualizations_data["player_name"]==player_names.value,'salary']
        with g.batch_update():
            g.data[0].x = x1
            g.data[0].y = y1
            #g.data[0].
            
            g.data[1].x=x1
            g.data[1].y=y1
            
            g.data[2].x=[.5 + visualizations_data.loc[visualizations_data["player_name"]==player_names.value, "season_start"].values[2], .5 + visualizations_data.loc[visualizations_data["player_name"]==player_names.value, "season_start"].values[2]]
            g.data[2].y=[-0.05* visualizations_data.loc[visualizations_data["player_name"]==player_names.value,'salary'].max(),    visualizations_data.loc[visualizations_data["player_name"]==player_names.value,'salary'].max() ]
            
            g.data[3].cells = dict( values=[visualizations_data.loc[ visualizations_data["player_name"]==player_names.value,col][0:1].tolist() for col in selected_cols], align = "center")
            
            g.update_layout(
    yaxis=dict(title="Salary"),
    xaxis=dict(title="Years"),
    template="plotly_dark",
    margin=dict(r=30, t=45, b=40, l=30),
    annotations=[
        go.layout.Annotation(
            text="BASKETBALL PLAYER'S DATA",
            showarrow=False,
            xref="paper",
            yref="paper",
            x=0,
            y=0),
        
        
        go.layout.Annotation(
            text="5 year Market Value :  $ "+str(round(visualizations_data.loc[visualizations_data["player_name"]==player_names.value, "salary"].values[-1-6:-1-1].sum()/ 1000000,2))+" M",
            showarrow=False,
            xref="paper",
            yref="paper",
            font=dict(family='Arial',size=16),
            x=0,
            y=.7)
    ]
)

            

player_names.observe(response, names="value")

In [9]:
# Generate the hbox for visual and vbox for player name selections
container2 = widgets.HBox([player_names])
widgets.VBox([player_names, g])